# Agents II: multiagent

* Multi-agent: Much the same way as a team can accomplish more than a single
 person, there are problems that can be best tackled by teams of LLM
 agents

### Subgraph

- Add a node that calls the subgraph directly
- Add a node with a function that invokes the subgraph

> Share by node

In [1]:
from langgraph.graph import START, StateGraph, END
from typing import TypedDict

class State(TypedDict):
    foo: str # this key is shared with the subgraph

class SubgraphState(TypedDict):
    foo: str # this key is shared with the parent graph
    bar: str

# Define subgraph
def subgraph_node(state: SubgraphState):
    # note that this subgraph node can communicate with the parent graph 
    # via the shared "foo" key
    return {"foo": state["foo"] + "bar"}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node("node_subgraph",subgraph_node)
subgraph_builder.add_edge(START, "node_subgraph")
subgraph = subgraph_builder.compile()

# Define parent graph
builder = StateGraph(State)
builder.add_node("subgraph", subgraph)
builder.add_edge(START, "subgraph")
builder.add_edge("subgraph", END)
graph = builder.compile()

In [2]:
input = {
  "foo":"Hi"
}
for c in graph.stream(input):
    print(c)

{'subgraph': {'foo': 'Hibar'}}


> Calling by function

In [7]:
class State(TypedDict):
    foo: str

class SubgraphState(TypedDict):
    # none of these keys are shared with the parent graph state
    bar: str
    baz: str

# Define subgraph
def subgraph_node(state: SubgraphState):
    return {"baz": state["baz"] + "bar"}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node("the_subgraph",subgraph_node)
subgraph_builder.add_edge(START, "the_subgraph")
subgraph = subgraph_builder.compile()

# Define parent graph
def node(state: State):
    # transform the state to the subgraph state
    response = subgraph.invoke({"baz": state["foo"]})
    # transform response back to the parent state
    return {"foo": response["baz"]}

builder = StateGraph(State)
# note that we are using `node` function instead of a compiled subgraph
builder.add_node("main",node)
builder.add_edge(START, "main")
builder.add_edge("main", END)
graph = builder.compile()

In [8]:
input = {
  "foo":"Hi"
}
for c in graph.stream(input):
    print(c)

{'main': {'foo': 'Hibar'}}


## Multi-agent

* Supervisor Architecture
* Network architecture
* Hierachical architecture
* Custom workflow

In [ ]:
from typing import Literal
from langchain_ollama import ChatOllama
from pydantic import BaseModel

class SupervisorDecision(BaseModel):
    next: Literal["researcher", "coder", "FINISH"]

model = ChatOllama(model="llama3.2:1b", temperature=0)
model = model.with_structured_output(SupervisorDecision)

agents = ["researcher", "coder"]

system_prompt_part_1 = f"""You are a supervisor tasked with managing a 
conversation between the following workers: {agents}. Given the following user 
request, respond with the worker to act next. Each worker will perform a
task and respond with their results and status. When finished,
respond with FINISH."""

system_prompt_part_2 = f"""Given the conversation above, who should act next? Or 
    should we FINISH? Select one of: {', '.join(agents)}, FINISH"""

def supervisor(state):
    messages = [
        ("system", system_prompt_part_1),
        *state["messages"],
        ("system", 	system_prompt_part_2)
    ]
    state["next"] = model.invoke(messages) 
    return state["next"]

In [35]:
from typing import Literal, Annotated, TypedDict
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

model = ChatOllama(model="llama3.2:1b", temperature=0)

class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    next: Literal["researcher", "coder", "FINISH"]

def researcher(state: AgentState):
    response = model.invoke("Researching...")
    return {"messages": [response]}

def coder(state: AgentState):
    response = model.invoke("Coding...")
    return {"messages": [response]}

builder = StateGraph(AgentState)
builder.add_node(supervisor)
builder.add_node(researcher)
builder.add_node(coder)

builder.add_edge(START, "supervisor")
# route to one of the agents or exit based on the supervisor's decision
builder.add_conditional_edges("supervisor", lambda state: state["next"])
builder.add_edge("researcher", "supervisor")
builder.add_edge("coder", "supervisor")

supervisor = builder.compile()

In [36]:
from langchain_core.messages import HumanMessage

for c in supervisor.stream(
    {
        "messages":[HumanMessage("I need the code for Grounding DINO")],
        "next":"FINISH"
    }
    ):
    print(c)

Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel branch:to:FINISH, ignoring it.


{'supervisor': None}


>> Pending search on langchain doc